In [ ]:
# Disable the IPython pager
# https://gist.github.com/minrk/7715212
from IPython.core import page
page.page = print

In [ ]:
# enable the %lprun magic
%load_ext line_profiler

# 05 - Performance

Make it Work, make it Right, Make it Fast.

In [ ]:
import math
import numpy as np
import numba 

In this exercise, we will be working with the [Root Mean Square Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation) calculation used to evaluate a trained model.

$RMSE = \sqrt{\frac{\sum_{t=1}^{N}{(\hat{y_t} - y_t)^2}}{N}}$

A "naive" python implementation could look like this

In [ ]:
def rmse(x, y):
    N = len(x)
    errors = 0
    
    for i in range(N):
        errors += (x[i] - y[i]) ** 2
    
    return math.sqrt(errors / N)

To test performance, let's generate random data

In [ ]:
N = 1_000_000  # number of observations

In [ ]:
X = np.random.randn(N)
Y = np.random.randn(N)

## Timing

First, it is importnat to measure how long things actually take and interpret that in the context of the overall workflow.

Use `%time` to measure the execution time of a given code snippet (or `%%time` when using more than one line)

In [ ]:
%time rmse(X, Y)

Sometimes, the execution time fluctuates or is different the first time (e.g. due to caching, initialization or other effectes)

`%timeit` runs the given code snippet several times and calculates the average executino time. Use `%%timeit` for a multi-line version of that

In [ ]:
%timeit rmse(X, Y)

## Profiling
To better understand where the time is spent, use the [python profiler](https://docs.python.org/3/library/profile.html) to get per-function timings

In [ ]:
%prun rmse(X, Y)

The output can also be visualized (e.g. using [SnakeViz](https://jiffyclub.github.io/snakeviz/)) 

The [Line Profiler](https://github.com/rkern/line_profiler) provides an even greater level of detail - line by line

In [ ]:
%lprun -f rmse rmse(X, Y)

The first observation is that most of the time is spent on line 6 (summing up values) and line 5 (iterating)

## Optimization

We can try operating on entire arrays instead, which is highly optimized in NumPy

In [ ]:
def rmse_numpy(x, y):
    N = len(x)
    errors = (x - y) ** 2
    mean = errors.sum() / N
    return math.sqrt(mean)

In [ ]:
%timeit rmse_numpy(X, Y)

Compare against the original function

In [ ]:
%timeit rmse(X, Y)

This is a clear win, but perhaps we can do even more?

In [ ]:
%lprun -f rmse_numpy rmse_numpy(X, Y)

## Compilation

For an even greater speedup, we can use [Numba](https://numba.pydata.org/) to compile the function to machine instructions which are faster to execute (think C/C++ or Java).

This is achieved by adding the [`@njit`](https://numba.pydata.org/numba-doc/latest/reference/jit-compilation.html#numba.jit) decorator (for "just-in_time" and "no python")

In [ ]:
@numba.njit
def rmse_numba(x, y):
    N = len(x)
    errors = (x - y) ** 2
    mean = errors.sum() / N
    return math.sqrt(mean)

The first time such a function will be caled, the compilation will take place, incresing the execution time

In [ ]:
%time rmse_numba(X, Y)

But all subsequent calls will be faster

In [ ]:
%timeit rmse_numba(X, Y)

The more complex the code under compilation, the stronger the effect will typically be (here, it's only a mild improvement)
                                                                                        
Why not compile everything with Numba?
* not all code can be compilerd: most basic python and NumPy can be, complicated code (classes, dynamic code) or other libraries (e.g. Pandas) or data structures are not supported (see [Intel SDC](https://github.com/IntelPython/sdc) for that) 
* we can [no longer use line_profiler](https://stackoverflow.com/questions/54545511/using-line-profiler-with-numba-jitted-functions) or [debugger](https://numba.pydata.org/numba-doc/latest/user/faq.html#can-i-debug-a-jitted-function)

## Library implementation

In [ ]:
from statsmodels.tools import eval_measures

In [ ]:
%timeit eval_measures.rmse(X, Y)

Similar performance but most likely much more robust implementation (special cases, tests) make that the preferred option